In [1]:
import tensorflow.compat.v1 as tf
# The version of the tensorflow is v-2.0. 
# But I am using commands/libraries from v-1.0. So disabling the v2 behavior.
tf.disable_v2_behavior()
tf.disable_eager_execution()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
import numpy as np
import os
import argparse

In [3]:
from basic_lm import BasicNeuralLM
from utils import DataLoader, SmallConfig, MediumConfig, LargeConfig

In [4]:
conf = SmallConfig()

In [5]:
data = DataLoader('toy.txt', conf.batch_size, conf.num_steps)

In [7]:
model = BasicNeuralLM(conf, data)

In [8]:
model

In [9]:
model.build_graph()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [10]:
def generate_sequence(model, seed, max_seq_len=45):
    '''
    :param model: model object which has had model.build_graph() already run
    :param seed: list of num_steps strings
    :return: list of strings
    '''
    assert len(seed) == model.conf.num_steps, "Error: seed is of incorrect length, must provide list of {} strings".format(model.conf.num_steps)
    seq = []
    for w in seed:
        if w not in model.data.vocab:
            seq.append(model.data.word2idx['<UNK>'])
        else:
            seq.append(model.data.word2idx[w])


    saver = tf.train.Saver()

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/checkpoint'))

        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)

        current = seq[-1]
        while model.data.idx2word[current] != '#' and len(seq) <= max_seq_len:
            # Lazy solution here where we just pass the input sequence and a matrix of zeros so its the right shape
            input_matrix = np.zeros([model.conf.batch_size, model.conf.num_steps])
            input_matrix[0, :] = seq[-model.conf.num_steps:]
            feed_dict = {model.train_input: input_matrix}
            current = sess.run(tf.argmax(tf.nn.softmax(model.logits), axis=1), feed_dict=feed_dict)
            seq.append(current[0])
            current = current[0]

    return [model.data.idx2word[idx] for idx in seq]

In [12]:
def train_model(model, visualize_embeddings=True):
    '''
    :param model: model object which has had model.build_graph() already run
    :param num_train_steps: number of steps to train model
    :return: n/a
    '''
    saver = tf.train.Saver()

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/checkpoint'))

        #if ckpt and ckpt.model_checkpoint_path:
        #    saver.restore(sess, ckpt.model_checkpoint_path)

        initial_step = model.global_step.eval()
        for i in range(initial_step + model.conf.num_epochs):
            epoch_loss = 0.0
            # Training loop for epoch
            train_loss = 0.0
            j = 0
            print(sess.run(model.global_step))
            for batch_X, batch_y in model.data.load_batches():
                feed_dict = {model.train_input: batch_X, model.train_labels: batch_y}
                loss_batch, _ = sess.run([model.loss, model.optimizer],
                                                feed_dict=feed_dict)
                train_loss += loss_batch
                epoch_loss += loss_batch
                j += 1
                if (j + 1) % model.conf.skip_step == 0:
                    train_loss = sum(train_loss)
                    print('Average loss at step {}: {:5.1f}'.format(j, train_loss / model.conf.skip_step))
                    train_loss = 0.0
                    saver.save(sess, 'checkpoints/language-model', j)
            # Validation loop for epoch
            val_loss = 0.0
            for batch_X, batch_y in model.data.load_batches(kind='validation'):
                feed_dict = {model.train_input: batch_X, model.train_labels: batch_y}
                loss_batch, _ = sess.run([model.loss, model.optimizer],
                                                  feed_dict=feed_dict)
                val_loss += loss_batch
            print("Average validation set loss: {}".format(sum(val_loss) / model.data.num_batches["validation"]))
            print("Epoch {} complete with average error of {} on training set".format(i+1, sum(epoch_loss) / model.data.num_batches["train"]))
            print(' '.join(generate_sequence(model, ['#']*(data.window-1)+['i'])))
            print(' '.join(['-']*35))

        if visualize_embeddings:
            final_embed_matrix = sess.run(model.embed_matrix)
            embedding_var = tf.Variable(final_embed_matrix[:1000], name='embedding')
            sess.run(embedding_var.initializer)

            config = projector.ProjectorConfig()
            summary_writer = tf.summary.FileWriter('processed')

            embedding = config.embeddings.add()
            embedding.tensor_name = embedding_var.name

            embedding.metadata_path = 'processed/vocab_1000.tsv'
            projector.visualize_embeddings(summary_writer, config)
            saver_embed = tf.train.Saver([embedding_var])
            saver_embed.save(sess, 'processed/model3.ckpt', 1)

In [13]:
train_model(model)

0
Average loss at step 1: 402.4
Average loss at step 3: 902.8
Average validation set loss: 870.1387128829956
Epoch 1 complete with average error of 879.9116296768188 on training set
INFO:tensorflow:Restoring parameters from checkpoints/language-model-3
# # # # # i , , , , , <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
5
Average loss at step 1: 201.8
Average loss at step 3: 333.6
Average validation set loss: 298.18833319004625
Epoch 2 complete with average error of 344.33881920576096 on training set
INFO:tensorflow:Restoring parameters from checkpoints/language-model-3
# # # # # i reason one of necessary although well socialist objectives european actually , sectors supported of up opportunities , , , , , of times occurred well debate money european few by

Average validation set loss: 1.1759959872579202
Epoch 17 complete with average error of 1.154503178782761 on training set
INFO:tensorflow:Restoring parameters from checkpoints/language-model-3
# # # # # i opinion opinion this second would <UNK> the failure parliament in hence , of <UNK> , <UNK> , <UNK> , the , the , the , the , the , the , the , the , the , the , the
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
85
Average loss at step 1:   0.5
Average loss at step 3:   1.2
Average validation set loss: 1.1252310938434675
Epoch 18 complete with average error of 1.1040675123222172 on training set
INFO:tensorflow:Restoring parameters from checkpoints/language-model-3
# # # # # i opinion opinion this second would <UNK> the failure parliament in hence , of <UNK> , <UNK> , <UNK> , the , the , the , the , the , the , the , the , the , the , the
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
90
Average loss at step 1:   0.5
Average loss at step 3:

# # # # # i opinion opinion this second would <UNK> the failure parliament in hence , of <UNK> , <UNK> , <UNK> , the , the , the , the , the , the , the , the , the , the , the
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
165
Average loss at step 1:   0.3
Average loss at step 3:   0.7
Average validation set loss: 0.694982943998184
Epoch 34 complete with average error of 0.6822090588975698 on training set
INFO:tensorflow:Restoring parameters from checkpoints/language-model-3
# # # # # i opinion opinion this second would <UNK> the failure parliament in hence , of <UNK> , <UNK> , <UNK> , the , the , the , the , the , the , the , the , the , the , the
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
170
Average loss at step 1:   0.3
Average loss at step 3:   0.7
Average validation set loss: 0.6796337051782757
Epoch 35 complete with average error of 0.6672463847789913 on training set
INFO:tensorflow:Restoring parameters from checkpoints/language

Average loss at step 3:   0.6
Average validation set loss: 0.5158542545395903
Epoch 50 complete with average error of 0.5077948444522917 on training set
INFO:tensorflow:Restoring parameters from checkpoints/language-model-3
# # # # # i opinion opinion this second would <UNK> the failure parliament in hence , of <UNK> , <UNK> , <UNK> , the , the , the , the , the , the , the , the , the , the , the
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


NameError: name 'projector' is not defined